<a href="https://colab.research.google.com/github/abimmost/SEED-generative-Ai-with-interview-questions/blob/full-contribute/exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install virtualenv
!virtualenv myenv
!./myenv/bin/pip install fastapi transformers typing pydantic uvicorn pyngrok diffusers langchain_community

In [ ]:
# @title
import sys
import os

# 1. Define the path to the site-packages directory
# You might need to change 'python3.10' based on the Colab runtime version
env_path = './myenv/lib/'
python_version = f"python{sys.version_info.major}.{sys.version_info.minor}"
site_packages_path = os.path.join(env_path, python_version, 'site-packages')

# 2. Add the path to Python's system path
if site_packages_path not in sys.path:
    sys.path.append(site_packages_path)
    print(f"Added {site_packages_path} to sys.path.")
else:
    print(f"{site_packages_path} is already in sys.path.")

**ALL EXERCISES**

In [ ]:
%%writefile app.py

from fastapi import FastAPI, UploadFile, File, HTTPException
from transformers import pipeline
from typing import Optional

# @title
app = FastAPI()

## EXERCISE 1: Hello LLM Endpoint

@app.post("/hello-llm")
def hello(text: str):
    hello_pipeline = pipeline("text-generation", model="distilgpt2")
    greeting = hello_pipeline(text)
    return {
        "text": text,
        "Greetings": greeting
    }

## EXERCISE 2: TEXT SUMMARIZER

@app.post("/summarize")
def summary(long_text):
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    summarized = summarizer(
        long_text,
        max_length=50,
        min_length=15,
        do_sample=False
    )
    return {
        "Long Text": long_text,
        "Summary": summarized
    }

## EXERCISE 3: SENTIMENT ANALYSIS

@app.post("/sentiment")
def sentiment(text: str):
    sentiment_pipeline = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")
    senti = sentiment_pipeline(text)

    return {
        "text": text,
        "sentiment": senti
    }

## EXERCISE 4: MULTIMODAL IMAGE CAPTIONING
# By uploading image
@app.post("/caption-image")
async def run_file(file:UploadFile=File(...)):
  from PIL import Image
  import io

  caption_pipeline = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")
  image_bytes = await file.read()
  image = Image.open(io.BytesIO(image_bytes))
  captioned_image = caption_pipeline(image)
  return {
      "file": file.filename,
      "caption": captioned_image
    }

# By sending image link
@app.post("/caption-image-link")
def caption_image_link(image_link: str):
  caption_pipeline = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")
  captioned_image = caption_pipeline(image_link)
  return {
    "file": image_link,
    "caption": captioned_image
  }

## EXERCISE 10: DIFFUSION MODEL - IMAGE GENERATION

@app.post("/image-generation")
def image_gen(prompt: str):
  from diffusers import DiffusionPipeline

  pipe = DiffusionPipeline.from_pretrained("stable-diffusion-v1-5/stable-diffusion-v1-5")
  image = pipe(prompt).images[0]
  return {
    "prompt": prompt,
    "image": image
  }

## EXERCISE 8: CHAIN MULTIPLE TOOLS WITH LANGCHAIN

@app.post("/chain-tools")
def chain_tools(question: str):
  from langchain_community.document_loaders import WikipediaLoader

  wiki_loader = WikipediaLoader(query=question, lang="en")
  summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
  summarized = summarizer(
      wiki_loader,
      max_length=500,
      min_length=100,
      do_sample=False
  )
  sentiment_pipeline = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")
  sentiment = sentiment_pipeline(summarized)

  return {
    "question": question,
    "answer": summarized,
    "sentiment": sentiment
  }


**SERVING FASTAPI**

In [ ]:
# @title
from pyngrok import ngrok
import uvicorn
import nest_asyncio

# Get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
auth_token = "33EuUmVjTDuTEVQeCW6lBNFcGQZ_4ZQGynoNCAXRdgFkKSaNi"

# Set the authtoken
ngrok.set_auth_token(auth_token)

# Apply nest_asyncio to allow nested event loops (required for Colab)
nest_asyncio.apply()

# 1. Start the ngrok tunnel on the port where Uvicorn will run
NGROK_TUNNEL = ngrok.connect(49000)
print("Public URL:", NGROK_TUNNEL.public_url+"/docs")

# 2. Run Uvicorn
# The loop will block the notebook, but the URL will be active.
# Use reload=False in a Colab environment.
uvicorn.run("app:app", host="0.0.0.0", port=49000, log_level="info")